Реализовано раздельное детектирование и треккинг объектов по уже готовым массивам из детекции, что позволяет подбирать архитектуру трекера и постобработки изолировано, без необходимости запуска предикта YOLO (ускорение подбора треккера и постобработки примерно в 100 раз и без траты лимита GPU в колаб) - также потенциально возможна генетика архитектуры постобработки и треккинга. Кроме того, реализован проброс касок и жилетов мимо трекера, что позволяет лучше трековать людей, а также не приводит к фильтрации касок и жилетов трекером, это позволяет лучше детектировать нарушения, избегать ложных фиксаций нарушений. Также реализовано восстановление в массиве информации нетрекованных боксов людей (которые трекер отфильтровал, отбросил).
В постобработке реализовано два подхода по подсчету людей а также вероятностный подход фиксации нарушений. Под вероятностным понимается отношение количества кадров в которых конкретный айди находится в каске и жилете к общему количеству кадров с этим айди (отношение больше минимального порога - нарушения нет).

## Загрузка датасэта

In [1]:
from google.colab import drive           # у кого автоматически не подключается гугл диск
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import files
%matplotlib inline

In [3]:
!pip install ultralytics
!pip install filterpy
!git clone https://github.com/noahcao/OC_SORT.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.5/513.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 23.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110473 sha256=ae95b06e09aa20ad2c62ac1120709709470efc95c0c47adc68b6905b2bce79f6
  Stored in directory: /root/.cache/pip/wheels/53/e6/de/a09ea01e923aaf88b9f8c7c44329e857b2c1a31901167e55e6
Successfully built filterpy
Cloning into 'OC_SORT'...
remote: Enumerating objects: 493, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 493 (delta 44), reused 74 (delta 42), pack-reused 409

In [4]:
%cd /content/OC_SORT
from trackers.ocsort_tracker.ocsort import OCSort

/content/OC_SORT


In [5]:
%cd /content/
!git clone https://github.com/belouspavel/yolov8.git

/content
Cloning into 'yolov8'...
remote: Enumerating objects: 572, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 572 (delta 27), reused 67 (delta 17), pack-reused 493
Receiving objects: 100% (572/572), 86.65 MiB | 30.28 MiB/s, done.
Resolving deltas: 100% (188/188), done.


In [6]:
%cd /content/yolov8
from yolov8_tools import *

/content/yolov8


## Обучение и сохранение детекции по набору видео
Эту часть кода можно (нужно) выполнить только один раз для одной модели. Потом можно закоментить, мы будем использовать готовые файлы

In [ ]:
# model = YOLO('yolov8s.pt') # инициализируем модель

In [ ]:
# model.train(data='/content/drive/MyDrive/dataset-v1.2/data_custom.yaml', epochs=1)  # Обучаем модель

In [ ]:
# from google.colab import files  # здесь сохраняем веса от обучения
# files.download('/content/runs/detect/train/weights/best.pt') 

In [ ]:
""" Эта функция позволяет пройтись по всем номерованным видео от start_vid до end_vid
в папке video_source используя модель model_in_path находящуюся в папке path_model и 
складывая фалы с боксами от детекции (без треккера) в эту же папку где сама модель"""

path_model = '/content/drive/MyDrive/best.pt/45/'           # папка с моделью куда все складываем 
model_in_path = 'best45.pt'                                 # модель
video_source = '/content/drive/MyDrive/dataset-v1.1/test/'  # видео для детекции

detect_videos(path_model, model_in_path, video_source, start_vid = 1, end_vid = 2)

## Трекккинг и постобработка

In [7]:
"""Эта функция позволяет пройтись по всем сохраненным ранее детекциям от start_vid до end_vid 
если они существуют в папке с моделью path_model оттрековать эти боксы, наложить на соответствующие
видео и записать результат постобработки в словари"""

path_model = '/content/drive/MyDrive/best.pt/45/'   ## Модель
tracker_path = 'ocsort00/'
video_source ='/content/drive/MyDrive/dataset-v1.1/test/'

tracker = OCSort(det_thresh = 0.49428431641933235, max_age = 7, 
                  min_hits = 7, iou_threshold = 0.6247364818234254, 
                  delta_t = 5, asso_func = 'iou', 
                  inertia = 0.6758806605183052, use_byte = True)

d_in1, d_out1, d_in2, d_out2, d_helmet, d_unif = track_on_detect(path_model, tracker_path, video_source, tracker, start_vid = 1)

print('Количество входящих на первом алгоритме (№ видео: кол-во): ', d_in1)
print('Количество выходящих на первом алгоритме(№ видео: кол-во): ', d_out1)
print('Количество входящих на втором алгоритме(№ видео: кол-во): ', d_in2)
print('Количество выходящих на втором алгоритме(№ видео: кол-во): ', d_out2)
print('Люди в каске 1, без каски 0 (№ видео: порядок от первого к последнему): ', d_helmet)
print('Люди в жилете 1, без жилета 0 (№ видео: порядок от первого к последнему): ', d_unif)

данные по видео 32: отсутствуют
данные по видео 33: отсутствуют
данные по видео 34: отсутствуют
данные по видео 35: отсутствуют
данные по видео 37: отсутствуют
данные по видео 41: отсутствуют
данные по видео 43: отсутствуют
Количество входящих на первом алгоритме (№ видео: кол-во):  {'1': 3, '2': 3, '3': 3, '4': 4, '5': 0, '6': 1, '7': 0, '8': 0, '9': 1, '10': 0, '11': 0, '12': 0, '13': 2, '14': 0, '15': 2, '16': 0, '17': 0, '18': 4, '19': 1, '20': 0, '21': 1, '22': 0, '23': 0, '24': 1, '25': 2, '26': 0, '27': 1, '28': 1, '29': 1, '30': 1, '31': 0, '32': 0, '33': 0, '34': 0, '35': 0, '36': 0, '37': 0, '38': 8, '39': 0, '40': 0, '41': 0, '42': 2, '43': 0}
Количество выходящих на первом алгоритме(№ видео: кол-во):  {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 4, '7': 4, '8': 5, '9': 1, '10': 2, '11': 3, '12': 4, '13': 4, '14': 3, '15': 0, '16': 3, '17': 5, '18': 0, '19': 0, '20': 0, '21': 1, '22': 3, '23': 1, '24': 0, '25': 0, '26': 1, '27': 1, '28': 1, '29': 1, '30': 3, '31': 2, '32': 

## Подсчет ошибок

In [8]:
#Истинные значения вошедших и вышедших на видео
dict_in_true =  {'1':3,'2':3,'3':3,'4':4,'5':0,'6':1,'7':0,'8':0,'9':1,'10':0,'11':0,'12':0,'13':2,'14':0,'15':2,'16':0,'17':0,'18':4,'19':1,'20':0,'21':1,'22':0,'23':0,'24':1,'25':2,'26':0,'27':1,'28':1,'29':1,'30':1,'31':0,'32':0,'33':0,'34':0,'35':0,'36':0,'37':0,'38':8,'39':0,'40':0,'41':0,'42':2,'43':0}  # 20 видео битое с невошедшим призраком (человек раздвоился)
dict_out_true = {'1':0,'2':0,'3':0,'4':0,'5':0,'6':4,'7':4,'8':5,'9':1,'10':2,'11':3,'12':4,'13':4,'14':3,'15':0,'16':4,'17':6,'18':0,'19':0,'20':0,'21':1,'22':3,'23':1,'24':0,'25':0,'26':1,'27':1,'28':1,'29':1,'30':3,'31':2,'32':0,'33':0,'34':0,'35':0,'36':0,'37':0,'38':7,'39':1,'40':1,'41':0,'42':0,'43':0}

In [9]:
def compare_dict(d1,d2): #функция сравнения словарей входящих/выходящих предсказанных с эталонным словарем
  dict_all ={}
  d_1 = set(d1.keys())
  d_2 = set(d2.keys())
  common_keys = d_1.intersection(d_2)
  
  if common_keys:
    dict_all = {k: d1[k] - d2[k] for k in common_keys}
  return dict_all

In [11]:
d_in_comp = compare_dict(d_in1, dict_in_true)
for key, value in d_in_comp.items(): 
  if value != 0:
    print(f'в видео {key} входящих {dict_in_true[key]} а не {d_in1[key]}')
print(f'суммарное количество ошибок первого алгоритма на входящих рассматриваемых видео {sum(abs(ele) for ele in d_in_comp.values())}')

d_out_comp = compare_dict(d_out1, dict_out_true)
for key, value in d_out_comp.items(): 
  if value != 0:
    print(f'в видео {key} выходящих {dict_out_true[key]} а не {d_out1[key]}')
print(f'суммарное количество ошибок первого алгоритма на выходящих рассматриваемых видео {sum(abs(ele) for ele in d_out_comp.values())}')

d_in_comp = compare_dict(d_in2, dict_in_true)
for key, value in d_in_comp.items(): 
  if value != 0:
    print(f'в видео {key} входящих {dict_in_true[key]} а не {d_in2[key]}')
print(f'суммарное количество ошибок второго алгоритма на входящих рассматриваемых видео {sum(abs(ele) for ele in d_in_comp.values())}')

d_out_comp = compare_dict(d_out2, dict_out_true)
for key, value in d_out_comp.items(): 
  if value != 0:
    print(f'в видео {key} выходящих {dict_out_true[key]} а не {d_out2[key]}')
print(f'суммарное количество ошибок второго алгоритма на выходящих рассматриваемых видео {sum(abs(ele) for ele in d_out_comp.values())}')

суммарное количество ошибок первого алгоритма на входящих рассматриваемых видео 0
в видео 17 выходящих 6 а не 5
в видео 16 выходящих 4 а не 3
суммарное количество ошибок первого алгоритма на выходящих рассматриваемых видео 2
суммарное количество ошибок второго алгоритма на входящих рассматриваемых видео 0
в видео 17 выходящих 6 а не 5
в видео 16 выходящих 4 а не 3
суммарное количество ошибок второго алгоритма на выходящих рассматриваемых видео 2


## Нарушения на видео

In [12]:
#Истинные значения нарушений на видео
d_true_helmet = {'1': [1, 1, 1], '2': [0, 1, 1], '3': [1, 0, 1], '4': [1, 0, 1, 1], '5': [], '6': [0, 0, 0, 0, 1], '7': [0, 0, 0, 0], '8': [0, 0, 0, 0, 0], '9': [0, 1], '10': [0, 0], '11': [0, 0, 0], '12': [0, 0, 0, 1], '13': [1, 0, 0, 0, 0, 0], '14': [1, 1, 1], '15': [0, 0], '16': [0, 0, 1, 0], '17': [0, 1, 0, 0, 0, 0], '18': [1, 1, 1, 1], '19': [0], '20': [], '21': [0,0], '22': [0, 1, 0], '23': [0], '24': [1], '25': [0, 0], '26': [0], '27': [0, 1], '28': [0, 1], '29': [0, 0], '30': [0, 0, 0, 0], '31': [1, 1], '32': [], '33': [], '34': [], '35': [], '36': [], '37': [], '38': [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1], '39': [1], '40': [0], '41': [], '42': [0, 0], '43': []} # 20 видео битое с невошедшим призраком (человек раздвоился)
d_true_unif = {'1': [1, 1, 1], '2': [1, 1, 1], '3': [1, 0, 1], '4': [1, 0, 1, 1], '5': [], '6': [0, 0, 0, 0, 1], '7': [1, 0, 1, 0], '8': [0, 0, 0, 0, 0], '9': [1, 1], '10': [1, 1], '11': [0, 0, 1], '12': [0, 0, 0, 1], '13': [1, 1, 0, 0, 1, 1], '14': [1, 1, 1], '15': [0, 0], '16': [1, 1, 1, 0], '17': [0, 1, 0, 0, 1, 1], '18': [1, 1, 0, 0], '19': [1], '20': [], '21': [0,0], '22': [0, 1, 0], '23': [0], '24': [1], '25': [0, 0], '26': [0], '27': [0, 1], '28': [0, 1], '29': [0, 0], '30': [0, 1, 1, 1], '31': [1, 1], '32': [], '33': [], '34': [], '35': [], '36': [], '37': [], '38': [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1], '39': [1], '40': [0], '41': [], '42': [0, 0], '43': []} # 20 видео битое с невошедшим призраком (человек раздвоился)

In [13]:
""" 
Эта функция выравнивает длину массива в нарушениях по каждому видео так что бы подать в метрику sklearn массивы одной длины
Делает она это следующим образом: 
- Создаются копии массивов с эталоном и предсказанием
- Изменению подвергаются только те массивы (их копии) по видео в которых количество айди предсказанных отличается от эталонного !!!(их всего 1-2)!!!
В других массивах ничего менять не нужно потому что порядок в котором айди попадают в предсказания совпадает с эталоном 
(проверено в ручную на нескольких экспериментах)
- Копии этих массивов сортируются от 1 к 0 и там где не хватает значений добавляются маркеры отсутсвия детекции или доп детекции в случае эталона
( двойки - это третий класс снижающий P R для наших двух нарушения/ненарушения) 
Таким образом мы выравниваем длины массивов


Идея заключается в том что нам нужно при вычислении метрик подать пары значений и главное подать ту пару где у нас происходит отличие предсказания от эталона
Т.е. если предсказание определило 3 человека в касках а реально их было 4, то мы добьем четвертое значение двойкой и не важно какой из них не задетектровался
!!!Ситуация при которой два человека в эталоне по форме определились предиктом как один одетый по форме также будет штрафоваться добавлением двойки к массиву т.к. 
в противном случае это могло быть и два человека одетых по разному (один правильно - а другой нет) и обработка этого могла не заметить!!! 

Все обратные логики также справедливы!

Подход в рамках имеющихся тестовых видео работает (проверялось вручную на нашей пособработке по всем видео)

Возможно для цели подбора треккера с максимальной P метрикой по единицам логику нужно перевернуть и сортировать от меньшего к большему, добивать единицами - !Вернуться!

Ниже после формулы приведена демонстрация
"""

def dict_fill(d_pred_, d_true_):  # функция которая выравнивает содержимое в предсказанном словаре с эталоно заполняя его 0
  d_pred_exp = d_pred_.copy()  # создаем новые словари
  d_true_exp = d_true_.copy()
  if len(d_pred_exp) != len(d_true_exp): # если длина значений словарей не совпадает делаем соритрировку от 1 к 0 и добиваем  значения словаря меньшео размера маркером отсутсвия детекции - 2
    d_pred_exp.sort(reverse = True)
    d_true_exp.sort(reverse = True)
    _list = [d_pred_exp,d_true_exp]
    for a in _list:
      a.extend([2] * (max(map(len, _list)) - len(a)))

  return d_pred_exp, d_true_exp

### Демонстрация!
d_pred = {'1': [1, 0, 0], '2': [1, 0, 0, 0, 1, 0],  '3': [1, 0],        '4': [1, 0, 1]} # 1 - одного одетого задетектил как двоих раздетых 2 - лишние треки в середине видео 3 - пропуск отсутсвие детекции в конце 4 -объеденил в середине
d_true = {'1': [1, 1],    '2': [1, 0, 1, 0],        '3': [1, 0, 0, 1],  '4': [1, 0, 0, 1]}
for i in range(1,5):
  d_pred[f'{i}'], d_true[f'{i}'] = dict_fill(d_pred[f'{i}'], d_true[f'{i}'])
print(d_pred)
print(d_true)

{'1': [1, 0, 0], '2': [1, 1, 0, 0, 0, 0], '3': [1, 0, 2, 2], '4': [1, 1, 0, 2]}
{'1': [1, 1, 2], '2': [1, 1, 0, 0, 2, 2], '3': [1, 1, 0, 0], '4': [1, 1, 0, 0]}


In [14]:
# т.к. словари при подгонке размерности видоизменяются создаем новые что бы не потерять старые 
d_true_helmet_exp = {str(n): [] for n in list(range(1,44))}
d_helmet_exp = {str(n): [] for n in list(range(1,44))}  
d_true_unif_exp = {str(n): [] for n in list(range(1,44))} 
d_unif_exp = {str(n): [] for n in list(range(1,44))} 

# Проходимся по всем словарям с нарушениями и эталонным словарям и выравниваем их
for N in range(1,44):
  d_helmet_exp[f'{N}'], d_true_helmet_exp[f'{N}'] = dict_fill(d_helmet[f'{N}'], d_true_helmet[f'{N}']) # выравниваем количество (длины массивов) человек для каждого видео в предсказаниях и эталонах (описание функции выше)
  d_unif_exp[f'{N}'], d_true_unif_exp[f'{N}'] = dict_fill(d_unif[f'{N}'], d_true_unif[f'{N}']) # выравниваем количество (длины массивов) человек для каждого видео в предсказаниях и эталонах (описание функции выше)

#превращаем значения словарей сначала в список списков а потом в вектор для подачи в склерн
d_true_helmet_list = [element for innerList in list(d_true_helmet_exp.values()) for element in innerList] 
d_helmet_list = [element for innerList in list(d_helmet_exp.values()) for element in innerList] 
d_true_unif_list = [element for innerList in list(d_true_unif_exp.values()) for element in innerList] 
d_unif_list = [element for innerList in list(d_unif_exp.values()) for element in innerList] 

In [15]:
precision, recall,_,_ = precision_recall_fscore_support(d_true_helmet_list, d_helmet_list)
precision2, recall2,_,_ = precision_recall_fscore_support(d_true_unif_list, d_unif_list)
print('R по каскам: ', round(recall[1],2))
print('P по каскам: ', round(precision[1],2))
print('R по униформе: ', round(recall2[1],2))
print('P по униформе: ', round(precision2[1],2))

R по каскам:  1.0
P по каскам:  1.0
R по униформе:  0.94
P по униформе:  1.0


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
